# Segmentación de cerebro

## Imports

### Instalación de librerías

In [1]:
!pip install monai nibabel torch torchvision

### Llamado de librerías

In [5]:
import monai
print(monai.__version__)

1.5.0


In [7]:
import os
import glob
import torch
import numpy as np
import nibabel as nib
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd,
    ToTensord, RandFlipd, RandRotate90d
)
from monai.data import Dataset, DataLoader
from monai.networks.nets import UNet
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.inferers import sliding_window_inference

Nota:
AddChanneld -> EnsureChannelFirstd
Fuente: https://github.com/Project-MONAI/MONAI/discussions/8147


## Lectura de los datos

In [14]:
data_dir_t1 = 'datosparaentrenar/datos-originales'
data_dir_seg = 'datosparaentrenar/aparc-aseg'
t1_paths = sorted(glob.glob(os.path.join(data_dir_t1, '*.nii.gz')))
seg_paths = sorted(glob.glob(os.path.join(data_dir_seg, '*.nii.gz')))

# Crear lista de diccionarios para MONAI
data_dicts = [{'image': img, 'label': seg} for img, seg in zip(t1_paths, seg_paths)]

#### Verifico que se hayan leido bien los datos


In [15]:
print(f"Número de estudios: {len(data_dicts)}")
print("Primer ejemplo:")
print(data_dicts[0])

Número de estudios: 10
Primer ejemplo:
{'image': 'datosparaentrenar/datos-originales\\1301_ses-AM_T1w.nii.gz', 'label': 'datosparaentrenar/aparc-aseg\\1301-aparcaseg-label.nii.gz'}


## Preparación de los dataset

In [16]:
train_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    RandFlipd(keys=['image', 'label'], prob=0.5, spatial_axis=0),
    RandRotate90d(keys=['image', 'label'], prob=0.5, spatial_axes=(0, 1)),
    ToTensord(keys=['image', 'label'])
])


train_ds = Dataset(data=data_dicts, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)

## Configuración

## Entrenamiento

## Test